In [16]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [17]:
X = pd.read_csv('../treated_data/df_merged.csv', index_col=0)
X_test = pd.read_csv('../treated_data/df_merged_test.csv', index_col=0)


with open('../treated_data/type_of_feature.json', 'r') as fp:
    type_of_feature = json.load(fp)

/home/mateus/anaconda3/envs/home-credit/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (137,138,149,151) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/mateus/anaconda3/envs/home-credit/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/mateus/anaconda3/envs/home-credit/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3155: DtypeWarning: Columns (136,137,148,150) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Static DF train

In [18]:
# Create static df
df_static = X[type_of_feature['static']]
df_static.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [13]:
df_static['CREDIT_INCOME_PERCENT'] = df_static['AMT_CREDIT'] / df_static['AMT_INCOME_TOTAL']
df_static['ANNUITY_INCOME_PERCENT'] = df_static['AMT_ANNUITY'] / df_static['AMT_INCOME_TOTAL']
df_static['CREDIT_TERM'] = df_static['AMT_ANNUITY'] / df_static['AMT_CREDIT']
df_static['DAYS_EMPLOYED_PERCENT'] = df_static['DAYS_EMPLOYED'].apply(lambda x: 0 if x==365243.00 else x) / df_static['DAYS_BIRTH']
domain_fields = ['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT']

In [19]:
# Positive days dont make sense
df_static.DAYS_EMPLOYED.replace(365243,np.nan, inplace=True)
# create dummies
df_static = pd.get_dummies(df_static)
# group by SK_ID_CURR
df_static = df_static.groupby('SK_ID_CURR').median()
# drop columns with too many nulls
df_static = df_static[df_static.columns[df_static.isnull().mean() < 0.3]]
#select features with greatest correlation with target
# df_static_grouped = df_static_grouped[df_static_grouped.corr().abs()['TARGET'].sort_values(ascending=False)[:51].index]


/home/mateus/anaconda3/envs/home-credit/lib/python3.8/site-packages/pandas/core/series.py:4506: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [21]:
df_static = df_static[df_static.corr().abs()['TARGET'].sort_values(ascending=False)[:201].index]


In [20]:
df_static.shape

(49650, 204)

In [24]:
df_static.to_csv('../treated_data/df_static.csv')

In [6]:
print(oi)

NameError: name 'oi' is not defined

In [ ]:
# Create static df
df_static_test = X_test[type_of_feature['static']]
df_static_test.dtypes.unique()

In [ ]:
type_of_feature['static'].remove(['TARGET'])

In [ ]:
# Positive days dont make sense
df_static_test.DAYS_EMPLOYED.replace(365243,np.nan, inplace=True)
# create dummies
df_static_test = pd.get_dummies(df_static_test)
# group by SK_ID_CURR
df_static_grouped_test = df_static_test.groupby('SK_ID_CURR').median()



# use the same columns as in train df
df_static_grouped_test = df_static_grouped_test[df_static_grouped.columns]
#fill na with median from train
df_static_grouped_test = df_static_grouped_test.fillna(df_static_grouped.median())
#fill na with ZERO
df_static_grouped_test = df_static_grouped_test.fillna(0)

In [ ]:
df_static_grouped_test.to_csv('../treated_data/df_static_test.csv')